In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import tqdm
from pandas import DataFrame as df
from os.path import join
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import ipycytoscape
import json

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
pass

In [33]:
with open("pass1.json") as fi:
    json_file = json.load(fi)

In [34]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(json_file, directed=True)

In [35]:
cytoscapeobj.set_layout(name='dagre', nodeSpacing=10, edgeLengthVal=10)
cytoscapeobj.set_style([
    {'selector': 'node','style': {
        'font-family': 'arial',
        'font-size': '10px',
        'label': 'data(label)',
        'background-color': 'green'}},
    
    {'selector': 'node[classes="inv"]','style': {
        'background-color': 'yellow'}},
    
     {'selector': 'node[classes="non_inv"]','style': {
        'background-color': 'blue'}}
])

In [36]:
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLengthVal': 10}, cytoscape_style=[{…